In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# pylint: disable=invalid-name, too-many-locals, too-many-arguments, no-member

import os
import importlib
import numpy as np
import tensorflow as tf
import texar as tx

from ctrl_gen_model import CtrlGenModel


config = importlib.import_module('config')

In [2]:
from texar.modules import WordEmbedder, UnidirectionalRNNEncoder, \
        MLPTransformConnector, AttentionRNNDecoder, \
        GumbelSoftmaxEmbeddingHelper, Conv1DClassifier
from texar.core import get_train_op
from texar.utils import collect_trainable_variables, get_batch_size

In [3]:
# Data
train_data = tx.data.MultiAlignedData(config.train_data)
val_data = tx.data.MultiAlignedData(config.val_data)
test_data = tx.data.MultiAlignedData(config.test_data)
vocab = train_data.vocab(0)

# Each training batch is used twice: once for updating the generator and
# once for updating the discriminator. Feedable data iterator is used for
# such case.
iterator = tx.data.FeedableDataIterator(
    {'train_g': train_data, 'train_d': train_data,
     'val': val_data, 'test': test_data})
batch = iterator.get_next()

In [4]:
# Model
gamma = tf.placeholder(dtype=tf.float32, shape=[], name='gamma')
lambda_g = tf.placeholder(dtype=tf.float32, shape=[], name='lambda_g')
model = CtrlGenModel(batch, vocab, gamma, lambda_g, config.model)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.


## inside model

In [21]:
self = model
inputs = batch

In [22]:
embedder = WordEmbedder(
    vocab_size=vocab.size,
    hparams=self._hparams.embedder)
encoder = UnidirectionalRNNEncoder(hparams=self._hparams.encoder)

In [23]:
vocab.size

9361

In [24]:
self._hparams.embedder

In [25]:
embedder.trainable_variables

[<tf.Variable 'word_embedder_2/word_embedder:0' shape=(9361, 100) dtype=float32_ref>]

In [26]:
config.model

{'dim_c': 200,
 'dim_z': 500,
 'embedder': {'dim': 100},
 'encoder': {'rnn_cell': {'type': 'GRUCell',
   'kwargs': {'num_units': 700},
   'dropout': {'input_keep_prob': 0.5}}},
 'decoder': {'rnn_cell': {'type': 'GRUCell',
   'kwargs': {'num_units': 700},
   'dropout': {'input_keep_prob': 0.5, 'output_keep_prob': 0.5}},
  'attention': {'type': 'BahdanauAttention',
   'kwargs': {'num_units': 700},
   'attention_layer_size': 700},
  'max_decoding_length_train': 21,
  'max_decoding_length_infer': 20},
 'classifier': {'kernel_size': [3, 4, 5],
  'filters': 128,
  'other_conv_kwargs': {'padding': 'same'},
  'dropout_conv': [1],
  'dropout_rate': 0.5,
  'num_dense_layers': 0,
  'num_classes': 1},
 'opt': {'optimizer': {'type': 'AdamOptimizer',
   'kwargs': {'learning_rate': 0.0005}}}}

In [27]:
embedder.dim

100

In [28]:
encoder.state_size

700

In [29]:
# text_ids for encoder, with BOS token removed
enc_text_ids = inputs['text_ids'][:, 1:]
enc_outputs, final_state = encoder(embedder(enc_text_ids),
                                   sequence_length=inputs['length']-1)
z = final_state[:, self._hparams.dim_c:]

In [30]:
iterator.initialize_dataset(sess)
gamma_ = 1.
lambda_g_ = 0.

iterator.restart_dataset(sess, 'val')

feed_dict = {
                iterator.handle: iterator.get_handle(sess, 'val'),
                gamma: gamma_,
                lambda_g: lambda_g_,
                tx.context.global_mode(): tf.estimator.ModeKeys.EVAL
            }

In [46]:
sess.run(enc_text_ids,feed_dict=feed_dict).shape

(64, 16)

In [47]:
sess.run(enc_outputs,feed_dict=feed_dict).shape

(64, 16, 700)

In [48]:
sess.run(final_state,feed_dict=feed_dict).shape

(64, 700)

In [133]:
sess.run(z,feed_dict=feed_dict).shape

(64, 500)

In [50]:
model._hparams.dim_c

200

In [51]:
# Encodes label
label_connector = MLPTransformConnector(self._hparams.dim_c)

# Gets the sentence representation: h = (c, z)
labels = tf.to_float(tf.reshape(inputs['labels'], [-1, 1]))
c = label_connector(labels)
c_ = label_connector(1 - labels)
h = tf.concat([c, z], 1)
h_ = tf.concat([c_, z], 1)

In [55]:
sess.run(labels,feed_dict=feed_dict)

array([[1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.]], dtype=float32)

In [61]:
sess.run(c,feed_dict=feed_dict)

array([[-0.14826657,  0.08401954, -0.0273709 , ..., -0.10147383,
        -0.08362819, -0.12628421],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.14826657,  0.08401954, -0.0273709 , ..., -0.10147383,
        -0.08362819, -0.12628421],
       ...,
       [-0.14826657,  0.08401954, -0.0273709 , ..., -0.10147383,
        -0.08362819, -0.12628421],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.14826657,  0.08401954, -0.0273709 , ..., -0.10147383,
        -0.08362819, -0.12628421]], dtype=float32)

In [60]:
sess.run(c_,feed_dict=feed_dict)

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.14826657,  0.08401954, -0.0273709 , ..., -0.10147383,
        -0.08362819, -0.12628421],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.14826657,  0.08401954, -0.0273709 , ..., -0.10147383,
        -0.08362819, -0.12628421]], dtype=float32)

In [159]:
sess.run(final_state,feed_dict=feed_dict).shape

(64, 700)

In [160]:
sess.run(h,feed_dict=feed_dict).shape

(64, 700)

In [144]:
sess.run(model.samples['original'],feed_dict=feed_dict)

array([[  7, 459,  29, ...,   8,   2,   0],
       [ 25, 293,  30, ...,   0,   0,   0],
       [612,  46,  11, ...,   4,   2,   0],
       ...,
       [ 21,  20,   6, ...,   0,   0,   0],
       [ 14,  11,  71, ...,   0,   0,   0],
       [  5,  17,  11, ...,   2,   0,   0]])

In [64]:
# Teacher-force decoding and the auto-encoding loss for G
decoder = AttentionRNNDecoder(
    memory=enc_outputs,
    memory_sequence_length=inputs['length']-1,
    cell_input_fn=lambda inputs, attention: inputs,
    vocab_size=vocab.size,
    hparams=self._hparams.decoder)

connector = MLPTransformConnector(decoder.state_size)

g_outputs, _, _ = decoder(
    initial_state=connector(h), inputs=inputs['text_ids'],
    embedding=embedder, sequence_length=inputs['length']-1)

loss_g_ae = tx.losses.sequence_sparse_softmax_cross_entropy(
    labels=inputs['text_ids'][:, 1:],
    logits=g_outputs.logits,
    sequence_length=inputs['length']-1,
    average_across_timesteps=True,
    sum_over_timesteps=False)

In [119]:
b = sess.run(g_outputs,feed_dict=feed_dict)

In [129]:
b.logits[1][0].shape

(9361,)

In [120]:
len(b)

5

In [73]:
b[0].shape

(64, 16, 9361)

In [74]:
b[1].shape

(64, 16)

In [75]:
b[2].shape

(64, 16, 700)

In [76]:
b[3].shape

(64, 16, 16)

In [77]:
b[4].shape

(64, 16, 700)

In [78]:
# Gumbel-softmax decoding, used in training
start_tokens = tf.ones_like(inputs['labels']) * vocab.bos_token_id
end_token = vocab.eos_token_id
gumbel_helper = GumbelSoftmaxEmbeddingHelper(
    embedder.embedding, start_tokens, end_token, gamma)

soft_outputs_, _, soft_length_, = decoder(
    helper=gumbel_helper, initial_state=connector(h_))

In [82]:
sess.run(start_tokens,feed_dict=feed_dict) #64个

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)

In [85]:
end_token

2

In [114]:
b = sess.run(soft_outputs_,feed_dict=feed_dict)

In [117]:
b.sample_id

array([[[7.59797240e-06, 1.03159418e-05, 1.98395683e-05, ...,
         3.64804087e-04, 2.03961054e-06, 1.05032559e-05],
        [8.14804225e-05, 3.46020424e-05, 1.04083292e-05, ...,
         2.92571121e-06, 9.70610836e-06, 1.37982797e-05],
        [7.53640779e-06, 2.58242744e-05, 2.12993491e-05, ...,
         8.60251475e-06, 2.15658383e-05, 3.90810164e-05],
        ...,
        [1.08514741e-05, 4.23368438e-05, 1.11591826e-05, ...,
         8.01864007e-06, 2.85283977e-06, 8.00902490e-06],
        [1.99825240e-06, 1.78916198e-05, 5.42382077e-06, ...,
         3.63696631e-06, 5.31261821e-06, 1.70881194e-05],
        [2.78724269e-06, 6.43636577e-06, 2.29557299e-05, ...,
         3.91974891e-06, 8.76599552e-06, 2.24746582e-05]],

       [[3.30221264e-05, 2.14687657e-06, 5.88184457e-06, ...,
         2.00939326e-06, 3.24353323e-06, 1.01313447e-06],
        [7.04760623e-06, 8.98932740e-06, 4.79178016e-06, ...,
         6.74654366e-05, 6.75009233e-06, 1.10910260e-05],
        [2.27829969e-05, 

In [89]:
len(b)

5

In [90]:
b[0].shape

(64, 20, 9361)

In [91]:
b[1].shape

(64, 20, 9361)

In [92]:
b[2].shape

(64, 20, 700)

In [93]:
b[3].shape

(64, 20, 16)

In [94]:
b[4].shape

(64, 20, 700)

In [95]:
sess.run(soft_length_,feed_dict=feed_dict)

array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 10, 20, 20, 20, 20, 20], dtype=int32)

In [96]:
# Greedy decoding, used in eval
outputs_, _, length_ = decoder(
    decoding_strategy='infer_greedy', initial_state=connector(h_),
    embedding=embedder, start_tokens=start_tokens, end_token=end_token)

In [99]:
b = sess.run(outputs_,feed_dict=feed_dict)

In [100]:
len(b)

5

In [101]:
b[0].shape

(64, 20, 9361)

In [103]:
sess.run(length_,feed_dict=feed_dict)

array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20], dtype=int32)

In [104]:
# Creates classifier
classifier = Conv1DClassifier(hparams=self._hparams.classifier)
clas_embedder = WordEmbedder(vocab_size=vocab.size,
                             hparams=self._hparams.embedder)

# Classification loss for the classifier
clas_logits, clas_preds = classifier(
    inputs=clas_embedder(ids=inputs['text_ids'][:, 1:]),
    sequence_length=inputs['length']-1)
loss_d_clas = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.to_float(inputs['labels']), logits=clas_logits)
loss_d_clas = tf.reduce_mean(loss_d_clas)
accu_d = tx.evals.accuracy(labels=inputs['labels'], preds=clas_preds)

# Classification loss for the generator, based on soft samples
soft_logits, soft_preds = classifier(
    inputs=clas_embedder(soft_ids=soft_outputs_.sample_id),
    sequence_length=soft_length_)
loss_g_clas = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.to_float(1-inputs['labels']), logits=soft_logits)
loss_g_clas = tf.reduce_mean(loss_g_clas)

# Accuracy on soft samples, for training progress monitoring
accu_g = tx.evals.accuracy(labels=1-inputs['labels'], preds=soft_preds)

# Accuracy on greedy-decoded samples, for training progress monitoring
_, gdy_preds = classifier(
    inputs=clas_embedder(ids=outputs_.sample_id),
    sequence_length=length_)
accu_g_gdy = tx.evals.accuracy(
    labels=1-inputs['labels'], preds=gdy_preds)

# Aggregates losses
loss_g = loss_g_ae + lambda_g * loss_g_clas
loss_d = loss_d_clas

In [112]:
sess.run(soft_logits,feed_dict=feed_dict)

array([0.01741586, 0.03394279, 0.02264042, 0.01497537, 0.01940306,
       0.01961852, 0.01133799, 0.02101284, 0.01516242, 0.01501904,
       0.00919292, 0.01450225, 0.01402471, 0.01561443, 0.00870784,
       0.01622221, 0.01207736, 0.01495792, 0.01874118, 0.00725839,
       0.01232787, 0.01560966, 0.01846682, 0.01064307, 0.01111605,
       0.01406671, 0.01103418, 0.01744304, 0.01826762, 0.00827709,
       0.01784866, 0.0159074 , 0.02273712, 0.02134864, 0.01100616,
       0.0170842 , 0.0185736 , 0.01049068, 0.01139423, 0.01997441,
       0.0153355 , 0.01649436, 0.01238613, 0.01190534, 0.01948663,
       0.0159913 , 0.00986294, 0.02123216, 0.0215549 , 0.01469933,
       0.01515879, 0.01348502, 0.02196291, 0.01476233, 0.00659403,
       0.02288788, 0.01546674, 0.02007548, 0.01291853, 0.01433617,
       0.0152465 , 0.02132571, 0.02042782, 0.00671127], dtype=float32)

In [113]:
sess.run(soft_preds,feed_dict=feed_dict)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [111]:
sess.run(accu_d,feed_dict=feed_dict)

0.5625

In [6]:
def _train_epoch(sess, gamma_, lambda_g_, epoch, verbose=True):
    avg_meters_d = tx.utils.AverageRecorder(size=10)
    avg_meters_g = tx.utils.AverageRecorder(size=10)

    step = 0
    while True:
        try:
            step += 1
            feed_dict = {
                iterator.handle: iterator.get_handle(sess, 'train_d'),
                gamma: gamma_,
                lambda_g: lambda_g_
            }

            vals_d = sess.run(model.fetches_train_d, feed_dict=feed_dict)
            avg_meters_d.add(vals_d)

            feed_dict = {
                iterator.handle: iterator.get_handle(sess, 'train_g'),
                gamma: gamma_,
                lambda_g: lambda_g_
            }
            vals_g = sess.run(model.fetches_train_g, feed_dict=feed_dict)
            avg_meters_g.add(vals_g)

            if verbose and (step == 1 or step % config.display == 0):
                print('step: {}, {}'.format(step, avg_meters_d.to_str(4)))
                print('step: {}, {}'.format(step, avg_meters_g.to_str(4)))

            if verbose and step % config.display_eval == 0:
                iterator.restart_dataset(sess, 'val')
                _eval_epoch(sess, gamma_, lambda_g_, epoch)

        except tf.errors.OutOfRangeError:
            print('epoch: {}, {}'.format(epoch, avg_meters_d.to_str(4)))
            print('epoch: {}, {}'.format(epoch, avg_meters_g.to_str(4)))
            break

def _eval_epoch(sess, gamma_, lambda_g_, epoch, val_or_test='val'):
    avg_meters = tx.utils.AverageRecorder()

    while True:
        try:
            feed_dict = {
                iterator.handle: iterator.get_handle(sess, val_or_test),
                gamma: gamma_,
                lambda_g: lambda_g_,
                tx.context.global_mode(): tf.estimator.ModeKeys.EVAL
            }

            vals = sess.run(model.fetches_eval, feed_dict=feed_dict)

            batch_size = vals.pop('batch_size')

            # Computes BLEU
            samples = tx.utils.dict_pop(vals, list(model.samples.keys()))
            hyps = tx.utils.map_ids_to_strs(samples['transferred'], vocab)
            print("samples: ",hyps)

            refs = tx.utils.map_ids_to_strs(samples['original'], vocab)
            refs = np.expand_dims(refs, axis=1)
            print("reference: ",refs)

            bleu = tx.evals.corpus_bleu_moses(refs, hyps)
            vals['bleu'] = bleu

            avg_meters.add(vals, weight=batch_size)

            ###################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # Writes samples
            tx.utils.write_paired_text(
                refs.squeeze(), hyps,
                os.path.join(config.sample_path, 'val.%d'%epoch),
                append=True, mode='v')

        except tf.errors.OutOfRangeError:
            print('{}: {}'.format(
                val_or_test, avg_meters.to_str(precision=4)))
            break

    return avg_meters.avg()

## restore model

In [ ]:
sess = tf.Session()

In [106]:
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [39]:
sess.run(tf.tables_initializer())

FailedPreconditionError: Table already initialized.
	 [[Node: train_2/key_value_init_1 = InitializeTableV2[Tkey=DT_STRING, Tval=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](train_2/hash_table_1, train/key_value_init/values, train/key_value_init/keys)]]

Caused by op 'train_2/key_value_init_1', defined at:
  File "/home/guojy/anaconda3/envs/pt4/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/guojy/anaconda3/envs/pt4/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/ho

In [7]:
saver = tf.train.Saver(max_to_keep=None)

In [8]:
saver.restore(sess, 'save2/model.ckpt')

INFO:tensorflow:Restoring parameters from save2/model.ckpt


In [20]:
var_name_list = [v.name for v in tf.trainable_variables()]

In [21]:
var_name_list

['word_embedder/word_embedder:0',
 'unidirectional_rnn_encoder/rnn/gru_cell/gates/kernel:0',
 'unidirectional_rnn_encoder/rnn/gru_cell/gates/bias:0',
 'unidirectional_rnn_encoder/rnn/gru_cell/candidate/kernel:0',
 'unidirectional_rnn_encoder/rnn/gru_cell/candidate/bias:0',
 'mlp_connector/dense/kernel:0',
 'mlp_connector/dense/bias:0',
 'attention_rnn_decoder/memory_layer/kernel:0',
 'mlp_connector_1/dense/kernel:0',
 'mlp_connector_1/dense/bias:0',
 'attention_rnn_decoder/decoder/attention_wrapper/gru_cell/gates/kernel:0',
 'attention_rnn_decoder/decoder/attention_wrapper/gru_cell/gates/bias:0',
 'attention_rnn_decoder/decoder/attention_wrapper/gru_cell/candidate/kernel:0',
 'attention_rnn_decoder/decoder/attention_wrapper/gru_cell/candidate/bias:0',
 'attention_rnn_decoder/decoder/attention_wrapper/bahdanau_attention/query_layer/kernel:0',
 'attention_rnn_decoder/decoder/attention_wrapper/bahdanau_attention/attention_v:0',
 'attention_rnn_decoder/decoder/attention_wrapper/attention_l

In [26]:
from tensorflow.python import pywrap_tensorflow

reader = pywrap_tensorflow.NewCheckpointReader('save2/model.ckpt')
var_to_shape_map = reader.get_variable_to_shape_map()

In [27]:
var_to_shape_map

{'unidirectional_rnn_encoder/rnn/gru_cell/candidate/bias': [700],
 'mlp_connector_1/dense/kernel': [700, 700],
 'mlp_connector/dense/kernel': [1, 200],
 'conv1d_classifier/conv_encoder/logit_layer/kernel': [384, 1],
 'mlp_connector/dense/bias': [200],
 'conv1d_classifier/conv_encoder/conv_pool_1/sequential_layer_1/conv_1_1/kernel': [4,
  100,
  128],
 'conv1d_classifier/conv_encoder/conv_pool_1/sequential_layer/conv_1/kernel': [3,
  100,
  128],
 'conv1d_classifier/conv_encoder/conv_pool_1/sequential_layer/conv_1/bias': [128],
 'attention_rnn_decoder/memory_layer/kernel': [700, 700],
 'attention_rnn_decoder/decoder/dense/bias': [9361],
 'attention_rnn_decoder/decoder/attention_wrapper/gru_cell/gates/kernel': [800,
  1400],
 'attention_rnn_decoder/decoder/attention_wrapper/gru_cell/gates/bias': [1400],
 'attention_rnn_decoder/decoder/attention_wrapper/gru_cell/candidate/kernel': [800,
  700],
 'attention_rnn_decoder/decoder/attention_wrapper/attention_layer/kernel': [1400,
  700],
 'Opt

## look up sentences

In [20]:
iterator.initialize_dataset(sess)
gamma_ = 1.
lambda_g_ = 0.

feed_dict = {
                iterator.handle: iterator.get_handle(sess, 'val'),
                gamma: gamma_,
                lambda_g: lambda_g_,
                tx.context.global_mode(): tf.estimator.ModeKeys.EVAL
            }

vals = sess.run(model.fetches_eval, feed_dict=feed_dict)

batch_size = vals.pop('batch_size')

# Computes BLEU
samples = tx.utils.dict_pop(vals, list(model.samples.keys()))
hyps = tx.utils.map_ids_to_strs(samples['transferred'], vocab)
print("samples: ",hyps)

refs = tx.utils.map_ids_to_strs(samples['original'], vocab)
refs = np.expand_dims(refs, axis=1)
print("reference: ",refs)

samples:  ['lots of nice home restaurant and etc ago !'
 'service was fast , friendly and professional .'
 'customer service was awesome , the food was fantastic .'
 "the room was fine , but i would n't sing about it ." ')'
 'the chicken screen was very tasteless and juicy .'
 'the staff was excellent and efficient .'
 "no server , i 'll just closed up the scallop ."
 'the staff is always friendly and willing to please .'
 'they lost our saturdays and our orders !'
 'love this place and we will go back !'
 "it 's still the best dressing food in phoenix !"
 "does n't make it fun ." 'the bathrooms was delicious .'
 'customer service is horrible .' 'incredible !'
 'also ordered the presentation and turkey techs which was very tasty .'
 'the stay at the personal bottle made it a little better .'
 'just enough people but small talented atmosphere .'
 'the locations were ok , chicken was modern sing to over eating it .'
 'food was delicious .' 'if your fries are hot , self yourself worked .'

In [24]:
len(hyps)

64

In [ ]:
 with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        sess.run(tf.tables_initializer())

        saver = tf.train.Saver(max_to_keep=None)
        if config.restore:
            print('Restore from: {}'.format(config.restore))
            saver.restore(sess, config.restore)

        iterator.initialize_dataset(sess)

        gamma_ = 1.
        lambda_g_ = 0.
        for epoch in range(1, config.max_nepochs+1):
            if epoch > config.pretrain_nepochs:
                # Anneals the gumbel-softmax temperature
                gamma_ = max(0.001, gamma_ * config.gamma_decay)
                lambda_g_ = config.lambda_g
            print('gamma: {}, lambda_g: {}'.format(gamma_, lambda_g_))

            # Train
            iterator.restart_dataset(sess, ['train_g', 'train_d'])
            _train_epoch(sess, gamma_, lambda_g_, epoch)

            # Val
            iterator.restart_dataset(sess, 'val')
            _eval_epoch(sess, gamma_, lambda_g_, epoch, 'val')

            #saver.save(sess, os.path.join(config.checkpoint_path, 'ckpt'), epoch)
            saver.save(sess, os.path.join('screensave', 'model'), epoch)

            # Test
            iterator.restart_dataset(sess, 'test')
            _eval_epoch(sess, gamma_, lambda_g_, epoch, 'test')

if __name__ == '__main__':
    tf.app.run(main=_main)
